In [19]:
import sys
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline

# the following line is not required if BatchFlow is installed as a python package.
sys.path.append('../../../batchflow')
from batchflow import Dataset, V, F, B, action, Batch
from batchflow.models.torch import TorchModel
from batchflow.models.metrics import ClassificationMetrics
from batchflow.opensets import MNIST
from batchflow import best_practice

In [67]:
from batchflow import B, V, F, R, P, C, action, inbatch_parallel, ImagesBatch

In [25]:
BATCH_SIZE = 64
mnist = MNIST()

Extracting /var/folders/d0/4xbr3ghj1nz4yr0d97gg0wkc0000gn/T/train-images-idx3-ubyte.gz
ExtractingDownloading  /var/folders/d0/4xbr3ghj1nz4yr0d97gg0wkc0000gn/T/train-labels-idx1-ubyte.gz
http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
ExtractingExtracting /var/folders/d0/4xbr3ghj1nz4yr0d97gg0wkc0000gn/T/t10k-labels-idx1-ubyte.gz
 /var/folders/d0/4xbr3ghj1nz4yr0d97gg0wkc0000gn/T/t10k-images-idx3-ubyte.gz


In [50]:
class RegressionModel(TorchModel):

    @classmethod
    def default_config(cls):
        config = TorchModel.default_config()
        config['head'].update(dict(layout='f'))
        return config

In [74]:
model_config = {
    'inputs': dict(images={'shape': (28, 28, 1)},
                   labels={'classes': 10}),
    'loss': 'ce',
    'optimizer': 'Adam',
    'initial_block/inputs': 'images',
    'head/units': 10,
    'output': dict(predicted=['proba', 'labels'])
}

In [81]:
data_dict = {
    'images': B('images'),
    'labels': B('labels')
}

In [88]:
train_logistic = (mnist.train.p
                .init_variable('predictions')
                .init_variable('loss_history', init_on_each_run=list)
                .init_model('dynamic',
                            RegressionModel,
                            'logistic',
                            config=model_config)
                .train_model('logistic',
                             fetches='predictions',
                             feed_dict=data_dict,
                             save_to=V('predictions'))
                .gather_metrics(ClassificationMetrics, targets=B('labels'), predictions=V('predictions'),
                    fmt='logits', axis=-1, save_to=V('metrics'), mode='a')
                .run(BATCH_SIZE, shuffle=True, n_epochs=10))

TypeError: train() got an unexpected keyword argument 'feed_dict'